In [ ]:
import sys

if '..' not in sys.path:
    sys.path.append('..')

# NSH

Computing and plotting the NSH velocity vectors

In [ ]:
from matplotlib import gridspec
from matplotlib.patches import Arc
import matplotlib.pyplot as plt
import numpy as np
import helper

In [ ]:
eps = 0.1
gamma = -2.75
eta = 0.001

all divided by `eta *vk`

## Combined plot

In [ ]:
fig = plt.figure(figsize=(15, 6), dpi=200)
gs = gridspec.GridSpec(2, 3, width_ratios=[2, 2, 1], hspace=0.05)

ax_c1 = fig.add_subplot(gs[0, 0])
ax_c2 = fig.add_subplot(gs[1, 0])

ax_v1 = fig.add_subplot(gs[:, 1])
ax_v2 = fig.add_subplot(gs[:, 2])

ax_c1.sharex(ax_c2)
ax_v1.sharey(ax_v2)

axc = [ax_c1, ax_c2]
axv = [ax_v1, ax_v2]


########## vector plot

delta = 1.13

ax_v1.set_xlim(-delta, 0)
ax_v2.set_xlim(0, delta/2)

for ax in axv:
    ax.set_ylim(-1.1 * delta, .1 * delta)
    
for ax in axv:
    ax.set_aspect('equal')
    # add horizontal line
    ax.axhline(0, c='k', lw=0.5)
    
pos_v1 = ax_v1.get_position()
pos_v2 = ax_v2.get_position()
ax_v2.set_position([pos_v1.x1, pos_v1.y0, pos_v2.width, pos_v2.height])



# arrow properties
props = {'width': 7e-3, 'length_includes_head':True}

leg_d = 'dust'
leg_g = 'gas'

for St in [0.01, 0.1, 1, 10]:
    
    # compute velocities in units of eta vk

    vdr = - 2 / (St + (1 + eps)**2 / St)
    vdp = - (1 + eps) / ((1 + eps)**2 + St**2)

    vgr = 2 * eps / (St + (1 + eps)**2 / St)
    vgp = (eps / (1 + eps) / (1 + (St / (1 + eps))**2) - 1)

    vd = np.sqrt(vdr**2 + vdp**2)
    vg = np.sqrt(vgr**2 + vgp**2)

    # plot the dust arrows
    
    #ax_v1.arrow(0, vdp, vdr,    0, **props, alpha=0.1)
    #ax_v1.arrow(0, 0,     0,  vdp, **props, alpha=0.1)
    ax_v1.arrow(0, 0,    vdr, vdp, **props, fc='C0', ec='C0', label=leg_d)
    
    # plot the gas arrows

    ax_v2.arrow(0, 0,  vgr, vgp, **props, fc='C2', ec='C2', label=leg_g)
    
    # label the dust vectors with `St = ...`
    
    fact = 1 + 0.10 / vd
    ax_v1.text(vdr * fact, vdp * fact, f'St = {St}',
            horizontalalignment='center', fontsize='medium', verticalalignment='center',
           color='C0', rotation=np.rad2deg(np.arctan(vdp/vdr)))

    leg_d = None
    leg_g = None
    
    # arc for St=1
    if St == 1:
        arc = Arc((0,0), 1, 1, angle=0.0, theta1=180.0, theta2=np.rad2deg(np.pi + np.arctan(0.5)))
        ax_v1.add_patch(arc)
        _r = 0.5
        ax_v1.plot([0, -_r], [0, -_r / 2], c='k')
        ax_v1.text(-0.45, -0.12, r'$\approx arctan(\mathsf{St}/2)$', rotation=13)
    

    # label last gas vector
    if St in [1, 10]:
        fact = 1 + 0.10 / vg
        ax_v2.text(vgr * fact, vgp * fact, f'St = {St}',
                horizontalalignment='center', fontsize='small', verticalalignment='center',
               color='C2', rotation=np.rad2deg(np.arctan(vgp/vgr)))
    
#### plot curves #### 

St = np.logspace(-2, 1, 100)
vdr = - 2 / (St + (1 + eps)**2 / St)
vdp = - (1 + eps) / ((1 + eps)**2 + St**2)
ax_v1.plot(vdr, vdp, 'C0', alpha=0.25)

vgr = 2 * eps / (St + (1 + eps)**2 / St)
vgp = (eps / (1 + eps) / (1 + (St / (1 + eps))**2) - 1)
ax_v2.plot(vgr, vgp, 'C2', alpha=0.25)


ax_v2.axes.get_yaxis().set_visible(False)
ax_v2.xaxis.set_ticks([0.25, 0.50])

texts = [fig.text(0.65, 0.05, 'radial velocity [$\eta V_\mathrm{k}$]', ha='center')]
texts += [ax_v1.set_ylabel('relative azimuthal velocity [$\eta V_\mathrm{k}$]')]
texts += [ax_v1.set_title('dust')]
texts += [ax_v2.set_title('gas')]

for t in texts:
    t.set_fontweight('bold')
    t.set_fontstyle('italic')

for ax in axv:
    helper.apply_araa_style(ax)

##################### Component Plot #####################

_St = np.logspace(-3, 2, 100)
ax_c1.set_xlim(_St[[0, -1]])

for i, epsilon in enumerate([0, 0.1, 1.0]):

# compute speeds

    vdr = - 2 / (_St + (1 + epsilon)**2 / _St)
    vdp = - (1 + epsilon) / ((1 + epsilon)**2 + _St**2)

    vgr = 2 * epsilon / (_St + (1 + epsilon)**2 / _St)
    vgp = (epsilon / (1 + epsilon) / (1 + (_St / (1 + epsilon))**2) - 1)
    

    # radial speeds

    ls = (i + 1) * '-'
    if i==2:
        ls=':'
    label = int(i==0)

    ax_c1.semilogx(_St, vdr, label=label * 'dust', c='C0', ls=ls)
    ax_c1.semilogx(_St, vgr, label=label * 'gas', c='C2', ls=ls)


    # Azimuthal speeds
    ax_c2.semilogx(_St, vdp, label=label * 'dust', c='C0', ls=ls)
    ax_c2.semilogx(_St, vgp, label=label * 'gas', c='C2', ls=ls)
    
    for ax in axc:
        ax.plot([], c='k', ls=ls, label=f'$\epsilon = {epsilon:.2f}$')

    
ax_c1.axvline(1, c='k', ls='--')
ax_c1.axhline(-1, c='k', ls='--')
ax_c1.set_ylabel(r'$\bf{radial}$ velocity [$\eta\,V_\mathrm{k}$]')
ax_c1.set_ylim(-1.1, 0.5)
#axs[0].legend();

ax_c2.axvline(1, c='k', ls='--')
ax_c2.axhline(-.5, c='k', ls='--')
ax_c2.set_xlabel('Stokes number')
ax_c2.set_ylabel(r'$\bf{relative azimuthal}$ velocity [$\eta\,V_\mathrm{k}$]')
ax_c2.set_ylim(-1.1, 0.1)
ax_c2.legend()

for ax in axc:
    helper.apply_araa_style(ax)
    
plt.setp(ax_c1.get_xticklabels(), visible=False);

In [ ]:
fig.savefig(helper.output_dir / 'BirnstielFig3.pdf', transparent=True, bbox_inches='tight')